<a href="https://colab.research.google.com/github/ryu-0729/ml-python/blob/master/chapter3/transfer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 転移学習

In [18]:
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

Colab only includes TensorFlow 2.x; %tensorflow_version has no effect.


#### 事前学習済みのInceptionモデル

In [19]:
import urllib.request
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop


weights_url = 'https://storage.googleapis.com/mledu-datasets/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'
weight_file = 'inception_v3.h5'
urllib.request.urlretrieve(weights_url, weight_file)

pre_trained_model = InceptionV3(
    input_shape=(150, 150, 3),
    include_top=False,
    weights=None,
)

pre_trained_model.load_weights(weight_file)

pre_trained_model.summary()


Model: "inception_v3"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 150, 150, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_94 (Conv2D)             (None, 74, 74, 32)   864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (None, 74, 74, 32)  96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                       

In [20]:
for layer in pre_trained_model.layers:
    layer.trainable = False

last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [21]:
# NOTE: 出力層を1次元に変形
x = layers.Flatten()(last_output)
# NOTE: ReLU活性化関数を指定して高密度に接続された1024個のニューロンを持つ隠れ層を追加
x = layers.Dense(1024, activation='relu')(x)
# NOTE: 分類のためのシグモイド層を最後に追加
x = layers.Dense(1, activation='sigmoid')(x)

#### モデル構築

In [22]:
model = Model(pre_trained_model.input, x)

model.compile(
    optimizer=RMSprop(lr=0.0001),
    loss='binary_crossentropy',
    metrics=['acc'],
)

In [23]:
# NOTE: 学習用データ
training_url = "https://storage.googleapis.com/learning-datasets/horse-or-human.zip"
training_file_name = "horse-or-human.zip"
training_dir = 'horse-or-human/training/'
urllib.request.urlretrieve(training_url, training_file_name)
zip_ref = zipfile.ZipFile(training_file_name, 'r')
zip_ref.extractall(training_dir)
zip_ref.close()

# NOTE: 検証用データ
validation_url = "https://storage.googleapis.com/learning-datasets/validation-horse-or-human.zip"
validation_file_name = "validation-horse-or-human.zip"
validation_dir = 'horse-or-human/validation/'
urllib.request.urlretrieve(validation_url, validation_file_name)

zip_ref = zipfile.ZipFile(validation_file_name, 'r')
zip_ref.extractall(validation_dir)
zip_ref.close()

In [29]:
# NOTE: 学習用
train_datagen = ImageDataGenerator(
    rescale=1./255.,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
)

train_generator = train_datagen.flow_from_directory(
    training_dir,
    batch_size=20,
    class_mode='binary',
    target_size=(150, 150),
)

# NOTE: 検証用
validation_datagen = ImageDataGenerator(rescale=1.0/255.)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    batch_size=20,
    class_mode='binary',
    target_size=(150, 150),
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


#### モデルの学習

In [30]:
history = model.fit_generator(
    train_generator,
    validation_data=validation_generator,
    epochs=20,
    verbose=1,
)

<ipython-input-30-ed6068f7a069>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20
52/52 [==============================] - 12s 228ms/step - loss: 0.1062 - acc: 0.9844 - val_loss: 0.0045 - val_acc: 1.0000
Epoch 2/20
52/52 [==============================] - 12s 234ms/step - loss: 0.0200 - acc: 0.9942 - val_loss: 2.0521e-04 - val_acc: 1.0000
Epoch 3/20
52/52 [==============================] - 12s 235ms/step - loss: 0.0508 - acc: 0.9922 - val_loss: 0.0120 - val_acc: 0.9922
Epoch 4/20
52/52 [==============================] - 12s 242ms/step - loss: 0.0442 - acc: 0.9903 - val_loss: 0.1642 - val_acc: 0.9766
Epoch 5/20
52/52 [==============================] - 12s 232ms/step - loss: 0.0072 - acc: 0.9971 - val_loss: 0.0242 - val_acc: 0.9961
Epoch 6/20
52/52 [==============================] - 12s 229ms/step - loss: 0.0304 - acc: 0.9961 - val_loss: 0.0914 - val_acc: 0.9805
Epoch 7/20
52/52 [==============================] - 12s 228ms/step - loss: 0.0471 - acc: 0.9883 - val_loss: 0.0107 - val_acc: 0.9961
Epoch 8/20
52/52 [==============================] - 12s 231ms/ste

In [36]:
import numpy as np
from google.colab import files
from keras.utils import load_img, img_to_array

uploaded = files.upload()

for fn in uploaded.keys():
  # predicting images
  path = '/content/' + fn
  img = load_img(path, target_size=(150, 150))
  x = img_to_array(img)
  x = np.expand_dims(x, axis=0)

  images = np.vstack([x])
  classes = model.predict(images)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is a human")
  else:
    print(fn + " is a horse")
 

Saving horse-g0054c7fc3_1920.jpg to horse-g0054c7fc3_1920 (1).jpg
Saving horse-ga29259509_1920.jpg to horse-ga29259509_1920 (7).jpg
1/1 [==============================] - 0s 48ms/step
[1.]
horse-g0054c7fc3_1920.jpg is a human
1/1 [==============================] - 0s 43ms/step
[1.]
horse-ga29259509_1920.jpg is a human
